In [1]:
import urllib.request
import pandas
import xml.etree.ElementTree as ET

import seaborn as sns
import pandas as pd
import numpy as np
from IPython.display import display
import matplotlib.pyplot as plt
from datetime import timedelta

# 한글 폰트 설정
from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)


# 파 가격 예측
## 데이터 로드

In [2]:
# 쿼리 보내는 함수
def query_sender(date, date_1, last_year, kinds):
    #url 보냅니다.
    url='http://www.garak.co.kr/publicdata/dataOpen.do?id=2972&passwd=qwert1324!&dataid=' \
        'data4&pagesize=10&pageidx=1&portal.templet=false&'

    kinds = urllib.parse.quote(kinds)
        # 아스키코드 형식이 아닌 글자를 URL 인코딩 시켜줌
        # kinds에는 농산물의 종류가 들어감(원래는 대파 말고 다른 것도 하려고 했습니다...)
    code_info='p_ymd=' + date+'&p_jymd='+date_1+'&d_cd=2&p_jjymd='+last_year
    code_info_2='&p_pos_gubun=9&pum_nm='+kinds
        # url에 들어갈 데이터와 string을 맞춤

    request = urllib.request.Request(url+code_info+code_info_2)
    request.get_method=lambda :'GET'
    response_body=urllib.request.urlopen(request).read()
    u=str(response_body,'utf-8')
        # 함수를 보내고 string으로 받아옴
    return u
        # 받아온 string 리턴


url에서 필요한 데이터를 수집해 오는 것
p_ymd: 검색일자, 8자리
y_jymd: 전일일자, 8자리
c_cd: 부류 구분(청과:2, 수산:3)
p_jjymd: 전년도 날짜, 8자리
p_pos_gubun: 시장구분, 가락시장:1, 양곡시장:2, 강서시장:3(경매), 강서시장:9(시장도매)
pum_nm: 품목명(예: 사과, 배)

In [3]:
# 받아온 아이템을 리스트로 전환해 주는 함수
def xml_to_item_list(xml_string, date):
    result=[]
        # 마지막에 리턴할 리스트
    root = ET.fromstring(xml_string)
        # 문자열에서 Element로 XML을 직접 구문 분석하는데, 구문 분석된 트리의 루트 엘리먼트
        # XML의 자료구조가 이렇게 되서 여기서 하나씩 불러서 쓰는 것 같음
    for child in root:
        item_list=[]
            # 리스트 한 row를 담당할 리스트
        item_list.append(date)
        for list in child:
            # 내가 원하는 정보를 가져오기
            if list.tag in ['PUM_NM_A', 'AV_P_A', 'PAV_P_A', 'PAV_PY_A', 'E_NAME']:
                item_list.append(list.text)
                # 한 줄 리스트에 정보 추가
        result.append(item_list)
            #한 줄씩 추출한 것을 result list에 붙이기
    return result

받아온 데이터를 리스트에 저장

한 줄의 정보는 item_list에 저장이 되고 전체 리스트는 result에 저장된다.
API 신청을 2021. 06. 10일까지 신청해서.. 지금은 안됩니다.. csv 파일도 첨부하겠습니다.

In [ ]:
flag=False
# csv 파일로 저장을 하기 위한 flag, 아래에서 파일이 있는지 없는지 확인하고,
# 파일이 없으면 새로 파일을 만들어 column name을 추가하고 데이터 넣기
# 생성된 파일이 있으면 그 아래 데이터를 추가하는 그런 형식으로 만들기 위한 flag

for date in pandas.date_range(start='20170101', end='20210527'):
    # 반복문으로 하세요

    # start와 end로 끝나는 반복문에서 입력타입이 String이므로 스트링으로 바꿔준다.
    # 반복문을 date_range로 한 이유는 숫자형으로 하면 20179999이런 식으로 갈 수 있기 때문이다.
    # date1: 오늘, date2: 전날, date3: 전년도 오늘
    date1 = str(date.strftime("%Y%m%d"))
    #print(date1)
    date2 = str((date-timedelta(days=1)).strftime("%Y%m%d"))
    date3 = str((date-timedelta(days=365)).strftime("%Y%m%d"))
    query=query_sender(date1,
                       date2,
                       date3,
                       '대파')
    #print("쿼리 = \n",query)
    # 함수를 호출한다.
    i_list=xml_to_item_list(query, date1)
    #print("i_list = \n", i_list)
    #display("리스트",i_list)

    df = pd.DataFrame(i_list)
    #df = df.iloc[1:, 1:]

    df=df.drop(df.index[0])
    #display(df)
    # csv파일에 저장할 때 첫번째 열에 column name을 넣어주기 위해 만약 파일이 없으면
    # 첫 번째 파일을 넣어주고 파일이 있으면 그 뒤로는 데이터를 계속 넣어준다.
    if not flag:
        flag = True
        col=['날짜', '품목명', '평균가격', '전일평균가격', '전년가격', '등급']
        df.columns=col
        df.to_csv('./data/Gonion.csv',encoding='utf-8-sig')
    else:
        df.to_csv('./data/Gonion.csv',encoding='utf-8-sig',mode='a', header=False)

print('끝났다!@@@@@@@@@@@@@@@!!!!!!!!!!!!!!!!!!!!!!@@@@@@@@@@')
# 시간이 오래 걸려서 데이터를 다 넣으면 나오는 출력문

테스트할 최신 데이터 다운

In [ ]:
flag=False
# 여기로 쿼리를 불러와서 확인하기!
for date in pandas.date_range(start='20210526', end='20210606'):

    date1 = str(date.strftime("%Y%m%d"))
    date2 = str((date-timedelta(days=1)).strftime("%Y%m%d"))
    date3 = str((date-timedelta(days=365)).strftime("%Y%m%d"))
    query1=query_sender(date1,
                       date2,
                       date3,
                       '대파')

    i_list1=xml_to_item_list(query1, date1)
    check = pd.DataFrame(i_list1)
    check=check.drop(check.index[0])
    if not flag:
        flag = True
        col=['날짜', '품목명', '평균가격', '전일평균가격', '전년가격', '등급']
        check.columns=col
        check.to_csv('./data/check.csv',encoding='utf-8-sig')
    else:
        check.to_csv('./data/check.csv',encoding='utf-8-sig',mode='a', header=False)

check = pd.read_csv('./data/check.csv', index_col=0)
display(check)